In [1]:
import json
import json_coder
import pandas as pd
import networkx as nx
import befaas as bf
import befaas.logentry as le
from befaas.graph import build_function_graph, add_default_metadata, conv_to_ms

filepath = "../dumps/"
logdumps = ["iot_aws_split.json","iot_aws_edge.json","iot_azure_split.json", "iot_azure_edge.json", "iot_google_split.json", "iot_google_edge.json"]
#logdumps = ["iot_aws_split.json","iot_aws_edge.json"]
#logdumps = ["iot_aws_split.json"]

outfile = "../img/functionRuntimesIoT.pkl"
plotdata = []

print("Everything is set up.")

Everything is set up.


In [2]:
for dump in logdumps:
    print(f"Include dump {dump} ...")

    platform = dump.split("_")[1]
    setup = dump.split("_")[2].split(".")[0]

    print("Load dump... (may take a while)")
    data = bf.load_logs(filepath + dump)
    print("Done.")

    print("Sort data to contexts...")
    contexts = {}

    for entry in data:
        id = entry.context_id
        if id == None:
            continue

        if not (id in contexts):
            contexts[id] = []

        contexts[id].append(entry)
    print("Done.")

    print("Find xpairs...")
    xpairs = {}

    for ctx_id in contexts:
        ctx = contexts[ctx_id]
        for entry in ctx:
            id = entry.event["xPair"]
            if not (id in xpairs):
                xpairs[id] = []

            specialPair = False
            if isinstance(entry, le.PerfLog):
                if not entry.perf_type_data == "":
                    parts = entry.perf_type_data.split(":")
                    if len(parts) > 1:
                        id = parts[1]
                        if not (id in xpairs):
                            xpairs[id] = []
                        xpairs[id].append(entry)
                        specialPair = True

            if not specialPair:
                xpairs[id].append(entry)
    print("done.")

    print("Sum up function execution times...")
    computeSum = 0
    networkSum = 0
    databaseSum = 0
    debugpairs = []

    for xpairID in xpairs:
        xpair = xpairs[xpairID]

        for entry in xpair:
                if isinstance(entry, le.PerfLog):
                    if entry.type == "measure":
                        if entry.perf_type[1] == "post":
                            computeSum = computeSum + entry.perf["duration"]
                        if entry.perf_type[1] == "rpcIn":
                            computeSum = computeSum + entry.perf["duration"]
                        if entry.perf_type[1] == "msg":
                            computeSum = computeSum + entry.perf["duration"]
                        if entry.perf_type[1] == "rpcOut":
                            networkSum = networkSum + entry.perf["duration"]
                            computeSum = computeSum - entry.perf["duration"]
                        if entry.perf_type[1] == "dbOut":
                            databaseSum = databaseSum + entry.perf["duration"]
                            computeSum = computeSum - entry.perf["duration"]

    # print(str(executionDurationSum) + " ms; " + str (networkLatencySum) + " ms; "  + str (databaseDurationSum) + " ms")
    # print(str(executionDurationSum/1000) + " s; " + str (networkLatencySum/1000) + " s; "  + str (databaseDurationSum/1000) + " s")
    # print(str(executionDurationSum/1000/60) + " min; " + str (networkLatencySum/1000/60) + " min; "  + str (databaseDurationSum/1000/60) + " min")

    row = {}
    row["platform"] = platform
    row["setup"] = setup
    row["kind"] = "a_database"
    row["value"] = databaseSum/1000
    row["valuec"] = databaseSum/1000
    plotdata.append(row)

    row = {}
    row["platform"] = platform
    row["setup"] = setup
    row["kind"] = "b_compute"
    row["value"] = computeSum/1000
    row["valuec"] = computeSum/1000 + databaseSum/1000
    plotdata.append(row)

    row = {}
    row["platform"] = platform
    row["setup"] = setup
    row["kind"] = "c_network"
    row["value"] = networkSum/1000
    row["valuec"] = networkSum/1000 + computeSum/1000 + databaseSum/1000
    plotdata.append(row)







Include dump iot_aws_split.json ...
Load dump... (may take a while)
Done.
Sort data to contexts...
Done.
Find xpairs...
done.
Sum up function execution times...
Include dump iot_aws_edge.json ...
Load dump... (may take a while)
Done.
Sort data to contexts...
Done.
Find xpairs...
done.
Sum up function execution times...
Include dump iot_azure_split.json ...
Load dump... (may take a while)
Done.
Sort data to contexts...
Done.
Find xpairs...
done.
Sum up function execution times...
Include dump iot_azure_edge.json ...
Load dump... (may take a while)
Done.
Sort data to contexts...
Done.
Find xpairs...
done.
Sum up function execution times...
Include dump iot_google_split.json ...
Load dump... (may take a while)
Done.
Sort data to contexts...
Done.
Find xpairs...
done.
Sum up function execution times...
Include dump iot_google_edge.json ...
Load dump... (may take a while)
Done.
Sort data to contexts...
Done.
Find xpairs...
done.
Sum up function execution times...


In [3]:
df_calls = pd.DataFrame(plotdata)
df_calls.head()
df_calls.describe()

,value,valuec
count,18.000000,18.000000
mean,655.666349,963.773852
std,444.022642,790.884097
min,27.680618,27.680618
25%,163.600788,163.600788
50%,697.469568,823.436466
75%,1079.321584,1891.943873
max,1249.620576,2044.879809


In [4]:
df_calls.to_pickle(outfile)